# Prompt Select

In [1]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

# 創建反義詞任務
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "hot", "output": "cold"},
    {"input": "up", "output": "down"},
    {"input": "big", "output": "small"},
    {"input": "fast", "output": "slow"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

examples_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=50,  # 設定最大長度
)

dynamic_prompt = FewShotPromptTemplate(
    example_selector=examples_selector,
    example_prompt=example_prompt,
    prefix="Please provide the antonym for the following word:\n",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
    )


In [2]:
print(dynamic_prompt.format(adjective="happy"))

Please provide the antonym for the following word:


Input: happy
Output: sad

Input: hot
Output: cold

Input: up
Output: down

Input: big
Output: small

Input: fast
Output: slow

Input: happy
Output:


In [3]:
long_text = "This is a long and short, big but small article."
print(dynamic_prompt.format(adjective=long_text))

Please provide the antonym for the following word:


Input: happy
Output: sad

Input: hot
Output: cold

Input: up
Output: down

Input: big
Output: small

Input: fast
Output: slow

Input: This is a long and short, big but small article.
Output:


In [4]:
new_examples = {"input": "light", "output": "heavy"}
dynamic_prompt.example_selector.add_example(new_examples)
print(dynamic_prompt.format(adjective="light"))

Please provide the antonym for the following word:


Input: happy
Output: sad

Input: hot
Output: cold

Input: up
Output: down

Input: big
Output: small

Input: fast
Output: slow

Input: light
Output: heavy

Input: light
Output:


In [5]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os
openai_api_key = os.getenv("EMPTY")
chat = ChatOpenAI(
    openai_api_key=openai_api_key,
    model="llama3.2",
    temperature=0.7,
    openai_api_base="http://127.0.0.1:11434/v1",
)

output_parser = StrOutputParser()

chain = dynamic_prompt | chat | output_parser

chain.invoke({"adjective": "light"})

'The antonym for the word "light" is actually "dark", not "heavy". Heavy and dark are related but distinct concepts.\n\nSo, the corrected output would be:\n\n Input: light\nOutput: dark'

In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.example_selector import (MaxMarginalRelevanceExampleSelector, SemanticSimilarityExampleSelector) 
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings_path = "D:\\ai\\downloads\\bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}"
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "hot", "output": "cold"},
    {"input": "up", "output": "down"},
    {"input": "big", "output": "small"},
    {"input": "fast", "output": "slow"},
]